# Section 2: Handling the label noise in datasets

Throughout this section, you will get familiar with:

* Why the label noise transition matrix $T$ is important in handling label noise?

* How do we estimate $T$ given a dataset with only noisy labels?

* How do we detect label errors in a dataset?

## 1. Importance of $T$

* Understanding the pattern/structure of label noise
* Design robust loss functions
* Helps label aggregation (weighted majority vote)

## 2. Estimate $T$

* Naive approach
* Estimate with anchor points
* Estimate with consensus patterns

## 2. Detect label errors

* Detect with model confidence
* Detect with sample influence
* Detect with similar features

Motivation for estimating T
Understanding (give a figure from CIFAR-N paper)
Give equation for loss correction (refer to next section)
Knowing T also helps aggregation (Sigmetrics’15)
Estimate T
Anchor point (equation only, no code)
HOC
Equation + Figure
Example
Load a X matrix 100*10
Load noisy Y (show ground truth T)
Find 2-NN (print an example)
Build tensor 
Solve equation 
Show transition matrix
Detection 
Confident learning (equation + intuition)
Influence function (def of influence function)
SimFeat
Equation + Figure
Example: 2D example
Show figure of the data points 
Use one wrongly labeled sample to show:
Find K-NN neighbor
Weighted majority vote
Ranking + HOC 
Show suggestion
